# Marathon Flag Model

Model using pandas

In [2]:
import pandas as pd

**Data set is from CHEVRON HOUSTON MARATHON 2019**

In [3]:
#Scraped Chicago marathon 2019
df = pd.read_csv('Chi_final.csv')

/Users/johnvictorifermo/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Code to find possible cheaters in Chevron Data set 

In [4]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Name,BIB,Age Group,Age,Place gender,Place Age,Place Overall,Start Time,...,40k Time of Day,40k Time,40k Difference,40k min/mile,Finish,Finish Time of Day,Finish Time.1,Finish Difference,Finish min/mile,sex
0,0,0,"Cherono, Lawrence (KEN)",4,30-34,31,1,1,1,07:30:03,...,09:29:12AM,01:59:10,15:17,04:55,Finish,09:35:47AM,02:05:45,06:36,04:50,M
1,1,1,"Debela, Dejene (ETH)",38,20-24,24,2,1,2,07:30:03,...,09:29:10AM,01:59:08,15:15,04:55,Finish,09:35:48AM,02:05:46,06:39,04:52,M
2,2,2,"Mengstu, Asefa (ETH)",5,30-34,31,3,2,3,07:30:03,...,09:29:10AM,01:59:08,15:15,04:55,Finish,09:35:50AM,02:05:48,06:40,04:53,M
3,3,3,"Karoki, Bedan (KEN)",9,25-29,29,4,1,4,07:30:03,...,09:29:11AM,01:59:09,15:16,04:55,Finish,09:35:55AM,02:05:53,06:45,04:57,M
4,4,4,"Abdi, Bashir (BEL)",10,30-34,30,5,3,5,07:30:03,...,09:29:55AM,01:59:53,14:39,04:43,Finish,09:36:16AM,02:06:14,06:21,04:40,M


In [5]:
#Remove uncessary column
df = df.drop(columns =['Unnamed: 0'])

#Pace's I need to convert
pacelist = ['5k min/mile','10k min/mile','15k min/mile','20k min/mile','Half min/mile','25k min/mile','30k min/mile','35k min/mile','40k min/mile','Finish min/mile']

#Quickly convert Pace to float
def paceconversion(x):
    for i in x:
        df[i]= df[i].str.replace(':','.')
paceconversion(pacelist)

#To make pace into floats
df[pacelist] = df[pacelist].apply(pd.to_numeric, downcast='float', errors='coerce')

#replace missing values(Missed timing matts) with Zero's
df = df.replace(to_replace ="–", value ="00.00")

# #Fill those with Nans
# df.fillna("00:00", inplace = True)
# df.isnull().sum()

#To find those with missing timing mats
missing = df[df.values  == "00.00"]


#To convert Finish time to datetime
def datetime(i):
    df[i] = pd.to_datetime(df[i]).dt.strftime('%H:%M:%S')
datetime('Finish Time.1')

#Make datetime column all into seconds
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*3600.0 + int(txt[1])*60.0 + int(txt[2])
    return ans

#Making a column of seconds so I can get bibmule cutoffs
df['Finish Time Seconds']= df['Finish Time.1'].apply(strtoseconds)

#To find bibmules
bibmule = df[df.duplicated(subset=['5k min/mile','10k min/mile','15k min/mile','20k min/mile','Half min/mile','25k min/mile','30k min/mile','35k min/mile','40k min/mile','Finish min/mile', 'Finish Time Seconds'], keep=False)]
#len(bibmule)

#Boston Qualifying time for Female 65 to 69 age groups Females is 4 hours 33 min 21 seconds
#This will be my bibmule cutoff
#(4*3600)+ (33*60) + 21 = 16401 seconds
bibmulecount = bibmule[(bibmule['Finish Time Seconds'] < 16401)]

#Weirdset
bibmule2 = df[df.duplicated(subset=['5k min/mile','10k min/mile','15k min/mile','Half min/mile','25k min/mile','30k min/mile','35k min/mile','40k min/mile','Finish min/mile'], keep=False)]
bibmulecount2 = bibmule2[(bibmule2['Finish Time Seconds'] < 16401)]

#Making a seperate dataframe to find course cutters
pacedf = df[['BIB','5k min/mile','10k min/mile','15k min/mile','Half min/mile','25k min/mile','30k min/mile','35k min/mile','40k min/mile','Finish min/mile']]

#Setting Index to BIB number
pacedf = pacedf.set_index('BIB')

#To find people who take shortcuts
shortcut = pacedf[(pacedf < 4.30).any(1)]

#Print results
print("Possible number of Marathoners to Flag in the Chicago Marathon.")
print("Initial findings show " + str(len(missing.BIB.unique())) + " have missed timing mats.")
print("There are " + str(len(bibmulecount)) + " possible bibmules.")
print("There are " + str(len(shortcut)) + " possible runners who took a shortcut.")

Possible number of Marathoners to Flag in the Chevron Marathon.
Initial findings show 85 have missed timing mats.
There are 132 possible bibmules.
There are 2 possible runners who took a shortcut.


**Use this site to reference for pictures**  
https://results.chicagomarathon.com/2019/?lang=EN_CAP&pid=start

In [7]:
#Print out list of marathoners who have missed timing mats
missed = missing.BIB.unique()
print("List of marathoners who have missed timing mats")
list(missed)

List of marathoners who have missed timing mats


[1130,
 9050,
 3209,
 17237,
 8253,
 20290,
 17769,
 6253,
 7795,
 888,
 16392,
 11518,
 24763,
 9825,
 18755,
 26913,
 26037,
 8888,
 39026,
 5850,
 28240,
 24700,
 7326,
 4798,
 35851,
 15270,
 33992,
 39273,
 24820,
 35125,
 21893,
 27086,
 28698,
 29259,
 39141,
 22692,
 9496,
 52308,
 36261,
 4835,
 24132,
 60328,
 28585,
 60619,
 3399,
 11638,
 51695,
 7410,
 25884,
 29951,
 31892,
 27792,
 27282,
 51010,
 11028,
 56300,
 47873,
 35357,
 47041,
 55660,
 37010,
 58768,
 24605,
 59351,
 46171,
 50248,
 63446,
 59559,
 43527,
 52320,
 34120,
 59735,
 46305,
 54995,
 62273,
 63150,
 50531,
 54619,
 60159,
 57525,
 51942,
 40637,
 46890,
 42468,
 51105]

In [8]:
#Print out list of marathoners who have possible bibmuled
print("List of Marathoners who may have possibly bibmuled")
list(bibmulecount.BIB)

List of Marathoners who may have possibly bibmuled


[2198,
 3005,
 13057,
 13041,
 13496,
 5368,
 6880,
 11121,
 12461,
 12041,
 20445,
 20451,
 18612,
 18068,
 3759,
 3760,
 21085,
 2148,
 21149,
 21156,
 20852,
 41627,
 30904,
 31082,
 10840,
 10838,
 7352,
 9845,
 14182,
 28182,
 5130,
 5129,
 53758,
 50795,
 18615,
 4994,
 36033,
 33209,
 5708,
 38228,
 40247,
 40248,
 27145,
 27143,
 7975,
 19554,
 34012,
 37445,
 6703,
 33085,
 24279,
 3840,
 32401,
 37570,
 29008,
 39486,
 17603,
 36022,
 18222,
 18213,
 38626,
 5701,
 3497,
 22557,
 36925,
 23575,
 4455,
 6577,
 35205,
 35064,
 2807,
 24525,
 27510,
 27539,
 19086,
 19085,
 36495,
 12995,
 36494,
 7830,
 23915,
 24552,
 39615,
 11954,
 36276,
 34251,
 30046,
 30006,
 10280,
 10953,
 34796,
 34788,
 35316,
 32321,
 16835,
 16834,
 32184,
 38329,
 18238,
 4116,
 36497,
 30428,
 39676,
 39675,
 56232,
 56231,
 47724,
 47726,
 40569,
 38872,
 47674,
 63566,
 25501,
 37386,
 4743,
 4742,
 38236,
 16012,
 30563,
 30564,
 1710,
 15747,
 26237,
 26244,
 53867,
 54958,
 29415,
 29414,
 4

In [9]:
print("List of possible people who took a short cut")
shortcut.head()

List of possible people who took a short cut


,5k min/mile,10k min/mile,15k min/mile,Half min/mile,25k min/mile,30k min/mile,35k min/mile,40k min/mile,Finish min/mile
BIB,,,,,,,,,
10048,7.24,7.27,2.46,7.51,7.54,7.57,8.03,8.21,8.24
30357,11.16,11.25,3.29,12.29,12.21,14.29,13.28,13.06,14.23


In [28]:
df1.head()

,Name,sex,BIB,Place Overall,Age Group,Age,Start Time,Finish Time of Day,Finish Time.1,Finish Time Seconds,...,10k min/mile,15k min/mile,20k min/mile,Half min/mile,25k min/mile,30k min/mile,35k min/mile,40k min/mile,Finish min/mile,Count
0,"Cherono, Lawrence (KEN)",M,4,1,30-34,31,07:30:03,09:35:47AM,02:05:45,7545.0,...,4.45,4.45,4.47,4.45,4.50,4.51,4.48,4.55,4.50,1
1,"Debela, Dejene (ETH)",M,38,2,20-24,24,07:30:03,09:35:48AM,02:05:46,7546.0,...,4.45,4.45,4.47,4.44,4.49,4.51,4.48,4.55,4.52,1
2,"Mengstu, Asefa (ETH)",M,5,3,30-34,31,07:30:03,09:35:50AM,02:05:48,7548.0,...,4.45,4.45,4.47,4.44,4.49,4.51,4.48,4.55,4.53,1
3,"Karoki, Bedan (KEN)",M,9,4,25-29,29,07:30:03,09:35:55AM,02:05:53,7553.0,...,4.44,4.45,4.47,4.44,4.49,4.52,4.48,4.55,4.57,1
4,"Abdi, Bashir (BEL)",M,10,5,30-34,30,07:30:03,09:36:16AM,02:06:14,7574.0,...,4.44,4.48,4.53,4.57,4.57,5.00,4.48,4.43,4.40,1


In [30]:
df1.to_csv('finalalpha.csv')

In [31]:
df1.tail()

,Name,sex,BIB,Place Overall,Age Group,Age,Start Time,Finish Time of Day,Finish Time.1,Finish Time Seconds,...,10k min/mile,15k min/mile,20k min/mile,Half min/mile,25k min/mile,30k min/mile,35k min/mile,40k min/mile,Finish min/mile,Count
44667,"Cook-Glanton, Regina (USA)",F,17383,45717,60-64,63,07:34:39,03:23:43PM,07:49:04,28144.0,...,16.059999,15.24,17.40,15.440000,19.379999,18.16,23.139999,20.170000,16.240000,1
44668,"Kalenga, Jane (ZAM)",F,908,45730,25-29,28,07:23:14,03:15:32PM,07:52:18,28338.0,...,17.270000,17.50,16.59,39.000000,18.580000,17.50,19.010000,17.370001,14.400000,1
44669,"Driscoll, Natasha (USA)",F,858,45748,45-49,45,07:23:16,03:22:26PM,07:59:10,28750.0,...,17.260000,17.50,17.00,38.580002,18.580000,17.50,19.010000,18.389999,17.200001,1
44670,"Byrnes, Lauren (USA)",F,855,45749,45-49,46,07:23:15,03:22:26PM,07:59:11,28751.0,...,17.240000,17.51,17.00,38.560001,18.590000,17.50,19.010000,18.379999,17.230000,1
44671,"Smith, Julie (USA)",F,862,45750,60-64,61,07:23:14,03:22:26PM,07:59:12,28752.0,...,17.240000,17.51,17.00,38.560001,18.580000,17.50,19.010000,18.379999,17.230000,1
